In [1]:
!pip install pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [25]:
!pip install huggingface_hub

In [2]:
import pandas as pd
from datasets import DatasetDict, Dataset

In [4]:
df = pd.read_csv("/content/phishing_site_urls.csv")

In [6]:
df = df.dropna()

In [7]:
df_safe = df[df['Label'] == 'good']
print(len(df_safe))
df_not_safe = df[df['Label'] == 'bad']
print(len(df_not_safe))

392924
156422


In [8]:
num_samples = 2000

In [9]:
df_safe_sample = df_safe.sample(num_samples, random_state = 42)
df_not_safe_sample = df_not_safe.sample(num_samples, random_state = 42)

In [10]:
df_safe_sample['Label'].unique()

array(['good'], dtype=object)

In [11]:
#replace "Email type" with boolean flag "isPhising"
df_safe_sample = df_safe_sample.assign(isPhising=False)
df_safe_sample = df_safe_sample.drop('Label', axis=1)

df_not_safe_sample = df_not_safe_sample.assign(isPhising=True)
df_not_safe_sample = df_not_safe_sample.drop('Label', axis=1)

In [12]:
balanced_df = pd.concat([df_safe_sample, df_not_safe_sample])

In [14]:
balanced_df.columns = ['text', 'labels']
balanced_df

,text,labels
313746,depositaccounts.com/savings/,False
305419,citypages.com/related/to/Dave+Simonett/,False
284414,askart.com/askart/c/kate_carew/kate_carew.aspx,False
153114,brianwattsphoto.com/,False
444475,thefreedictionary.com/action+deferred,False
...,...,...
516703,twitter.com/yO6fMyyspVLv,True
492104,titiuepmoi-macewicz.barchief.com/civis/index.p...,True
821,www.hadinhduc.com/PayPal/,True
487674,deployJava.js,True


In [15]:
balanced_df['labels'] = balanced_df['labels'].astype(int)

In [18]:
#Shuffle the balanced dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [19]:
train_frac = 0.7
valid_frac = 0.15
test_frac = 0.15

In [20]:
train_size = int(train_frac * len(balanced_df))
valid_size = int(valid_frac * len(balanced_df))

In [21]:
train_df = balanced_df[:train_size]
valid_df = balanced_df[train_size : train_size + valid_size]
test_df = balanced_df[train_size + valid_size :]

In [22]:
#Convert the pandas dataframe to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

In [23]:
dataset_dict = DatasetDict(
    {'train' : train_ds,
     'validation' : valid_ds,
     'test' : test_ds}
)

In [24]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2800
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 600
    })
})

In [26]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [27]:
dataset_dict.push_to_hub("agraj07/phishing_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/agraj07/phishing_dataset/commit/6a615ef457234f57f2562551ce0ad5798050970d', commit_message='Upload dataset', commit_description='', oid='6a615ef457234f57f2562551ce0ad5798050970d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/agraj07/phishing_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='agraj07/phishing_dataset'), pr_revision=None, pr_num=None)